## Generating Synthetic Seismogram in Python

Authors: Hyunmin Kim, Honggeun Jo | Modified from [Ryan A. Mardani](https://www.linkedin.com/in/amardani/)

In [1]:
import numpy as np
import pyvista as pv
# !pip install pyvista
# !pip install trame
# !pip install --upgrade trame-vuetify
# !pip install --upgrade trame-vtk
# !pip install ipywidgets

In [2]:
# step 0 - top depth and v_above given + other inputs
NX, NY, NZ = 64, 64, 32
tops = np.ones((NY, NX)) * 1500 # <- random input, meter flat horizon 
v_above = 1520                  # <- random input, m/s 
DZ = np.ones((32, 64, 64))      

# time domain variables
dt = 0.001   #sampleing interval
t_max = 3.0   # max time to create time vector
t = np.arange(0, t_max, dt)

# define Ricker wavelet
def ricker(f, length, dt):
    t0 = np.arange(-length/2, (length-dt)/2, dt)
    y = (1.0 - 2.0*(np.pi**2)*(f**2)*(t0**2)) * np.exp(-(np.pi**2)*(f**2)*(t0**2))
    return t0, y
f=20            # wavelet frequency
length=0.512    # wavelet vector length
dt=dt           # sampling prefer to use smiliar to resampled AI
t0, w = ricker(f, length, dt) # ricker wavelet 

In [3]:
# step 1 - import Vp and Rho_b, which will be imported from CMG runs
# saturation (composition), pressure, porosity (static/updated)
# TODO: convert S, P, Poro to Vp and Rho_b

Vp = np.random.uniform(1500, 2500, size = (32, 64, 64))
Rho_b = np.random.uniform(800, 1500, size = (32, 64, 64))

assert Rho_b.min() > 0 and Vp.min() > 0, "unrealistic input"

In [4]:
# step 2 - compute AI and TWT
AI = Vp * Rho_b
TWT = -np.ones_like(AI)
for i in range(NX):
    for j in range(NY):
        for k in range(NZ):
            twt = 0
            for s in range(k+1):
                twt += 2*DZ[s, j, i]/Vp[s, j, i]
            TWT[k, j, i] = 2*(tops[j,i]/v_above) + twt

assert TWT.min() > 0, 'negative TWT is detected'

In [6]:
# step 3 - resample AI in time domain
AI_tdom = -np.ones((t.shape[0], NY, NX))
for i in range(NX):
    for j in range(NY):
       AI_tdom[:, j, i] = np.interp(x=t, xp=TWT[:,j,i], fp = AI[:,j,i])

assert AI_tdom.min() > 0, 'negative AI is detected'

In [7]:
# step 4 - compute reflectivity in time domain
Rc_tdom = -np.ones_like(AI_tdom)
Rc_tdom[:-1] = (AI_tdom[1:]-AI_tdom[:-1])/(AI_tdom[1:]+AI_tdom[:-1]) 
Rc_tdom[-1] = Rc_tdom[-2]


In [10]:
# step 5 - convolve with the AI in time domain
seismic_tdom = -100*np.ones_like(AI_tdom)
for i in range(NX):
    for j in range(NY):
        seismic_tdom[:, j, i] = np.convolve(w, Rc_tdom[:, j, i], mode='same')

assert seismic_tdom.min() > -5 and seismic_tdom.max() < 5 , 'erroneous seismic is detected, seismic should be oscillated around zero'

In [12]:
# step 6 - resample seismic in depth domain
seismic = -np.ones_like(AI)
for i in range(NX):
    for j in range(NY):
        seismic[:, j, i] = np.interp(x=TWT[:, j, i], xp = t, fp = seismic_tdom[:, j, i])

In [13]:
# visualization 
grid = pv.ImageData()
grid.dimensions = np.array(seismic.T.shape) + 1
grid.origin = (1, 1, 1)  # The bottom left corner of the data set
grid.spacing = (1, 1, 1)  # These are the cell sizes along each axis
grid.cell_data["values"] = seismic.T.flatten(order="F")  # Flatten the array

grid.plot(show_edges=True, cmap = 'seismic')

Widget(value="<iframe src='http://localhost:62416/index.html?ui=P_0x127b1119b50_0&reconnect=auto' style='width…

In [22]:
seismic_tdom_cut = seismic_tdom[seismic_tdom.max(axis = 1).max(axis = 1) > 0.1]

In [25]:
seismic_tdom_cut.shape[0]/seismic_tdom_cut.shape[1]

1.1875

In [26]:
# visualization 
grid = pv.ImageData()
grid.dimensions = np.array(seismic_tdom_cut.T.shape) + 1
grid.origin = (1, 1, 1)  # The bottom left corner of the data set
grid.spacing = (1, 1, 1/(seismic_tdom_cut.shape[0]/seismic_tdom_cut.shape[1])/2)  # These are the cell sizes along each axis
grid.cell_data["values"] = seismic_tdom_cut.T.flatten(order="F")  # Flatten the array

grid.plot(show_edges=False, cmap = 'seismic')

Widget(value="<iframe src='http://localhost:62416/index.html?ui=P_0x127b3f03f10_5&reconnect=auto' style='width…

In [10]:
## TODO: Refer to https://docs.pyvista.org/version/stable/examples/00-load/terrain-mesh#sphx-glr-examples-00-load-terrain-mesh-py

### Refrences:<br>
[Agile geoscience and SEG WiKI](https://wiki.seg.org/wiki/Well_tie_calculus)